## Regression Modeling and Evaluation

* ##### Eunji Hwang (22100809)

### Problem Overview  
- #### Implementation of the Multi-Layer Perceptron (MLP) with the Backpropagation algorithm for addressing a regression problem

### 1) Import Libraries & load data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('20190120_Time_8_Altitude_22_Eastward_wind.csv')


/Users/hwang-eunji/opt/anaconda3/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


### 2) Data Pre-processing 

In [2]:
data.isnull().sum().sum() # checking for NA

0

In [3]:
data.dtypes # checking for data types

Longitude (deg)          int64
Latitude (deg)           int64
Eastward wind (m/s)    float64
dtype: object

In [4]:
# Convert Data type into float 32 
data['Longitude (deg)'] = data['Longitude (deg)'].astype(np.float32)
data['Latitude (deg)'] = data['Latitude (deg)'].astype(np.float32)

In [5]:
# Divide with Target and Feature
# x: feature varialbe
# y: target variable 
x = data[['Longitude (deg)', 'Latitude (deg)']].values 
y = data['Eastward wind (m/s)'].values.reshape(-1, 1)  # transform into colum vector


In [6]:
# Create a StandardScaler instance to scale attribute data
scaler = StandardScaler()

# Apply scaler to data x, y
X_scaled = scaler.fit_transform(x)
y_scaled = scaler.fit_transform(y)

In [7]:
# Split training data and test data, 20% of the total data is the test data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)



# Converting training data and test data in the form of Numpy arrays to the form of PyTorch's tensor
X_train = torch.tensor(X_train, dtype=torch.float32) 
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

### 3) Implement MLP

In [8]:
# Define MLP model classes
class MLP(nn.Module):
    def __init__(self):      # Initialization
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, 64),   # From the input-layer to the 1st hidden-layer, 2 input characteristics, 64 output neurons
            nn.ReLU(),          # First hidden-layer activation function ReLU
            nn.Linear(64, 128), # From the first-hidden layer to the 2nd hidden-layer, 64 input neurons, 128 output neurons
            nn.ReLU(),          # Second hidden-layer activation function ReLU
            nn.Linear(128, 1)   # From the second hidden-layer to the output layer, 128 input neurons, 1 output neuron (predicted value)
        )
    
    def forward(self, x):
        return self.layers(x)   # Calculate the final output through sequential layers given input x

# Instantiate the model
model = MLP()
criterion = nn.MSELoss()  # Mean squared error loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam Optimization Algorithm, Learning Rate 0.01

In [9]:
# Trainiing loop
for epoch in range(100):
    model.train()             # Set the model in training mode
    optimizer.zero_grad()     # Initialize gradient to zero
    outputs = model(X_train)  # Model to make predictions on training data
    loss = criterion(outputs, y_train)  # Calculate the loss of predicted and actual values
    loss.backward()      # Calculate the gradient based on the loss function
    optimizer.step()     # Update the weight of the model using the calculated gradient
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')  # Output current loss every 10 epoxes

Epoch 1, Loss: 1.0457535982131958
Epoch 11, Loss: 0.3859861493110657
Epoch 21, Loss: 0.22093407809734344
Epoch 31, Loss: 0.14823555946350098
Epoch 41, Loss: 0.11078191548585892
Epoch 51, Loss: 0.09195457398891449
Epoch 61, Loss: 0.08349984139204025
Epoch 71, Loss: 0.06036968529224396
Epoch 81, Loss: 0.050022851675748825
Epoch 91, Loss: 0.04150298610329628


### 4) Evaluation

In [10]:
# Define funtion to calcuate MSE
def calculate_mse(y_true, y_pred):  # actual value, predicted value
    return ((y_true - y_pred) ** 2).mean()

# Define funtion to calculate R-square 
def calculate_r_squared(y_true, y_pred): # actual value, predicted value
    ss_res = ((y_true - y_pred) ** 2).sum()
    ss_tot = ((y_true - y_true.mean()) ** 2).sum()
    return 1 - ss_res / ss_tot

# evaluation
model.eval()  # setting mode
with torch.no_grad():
    predictions = model(X_test) 
    mse = calculate_mse(y_test.numpy(), predictions.numpy())   # calculate MSE
    r_squared = calculate_r_squared(y_test.numpy(), predictions.numpy())  # calculate R-square

print(f'Calculated Test MSE: {mse:.4f}')
print(f'Calculated R-squared: {r_squared:.4f}')


Calculated Test MSE: 0.0372
Calculated R-squared: 0.9620


### 5) Visuallization  
$$\boldsymbol{x_1}\cdot \boldsymbol{x_2} \neq 0 $$

In [ ]:
model.eval()   # Set to Evaluation Mode

with torch.no_grad():  # Disable automatic differentiation 
    predictions = model(X_test).flatten()  # Perform predictions and make them a one-dimensional array


In [ ]:
# Visuzlize the Plot of Actual vs Predicted'
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5)) # size of graph
plt.scatter(y_test.numpy(), predictions.numpy(), alpha=0.5) 
plt.title('Actual vs Predicted')
plt.xlabel('Actual Eastward Wind')
plt.ylabel('Predicted Eastward Wind')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red')  # Diagonal line
plt.show()


In [ ]:
# Flatten arrays to ensure they are 1-D
predictions_flat = predictions.numpy().flatten()
y_test_flat = y_test.numpy().flatten()

# Calculate residuals
residuals = y_test_flat - predictions_flat

# plotting scatter graph of Residuals vs Predicted
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.scatter(predictions_flat, residuals, alpha=0.5)
plt.title('Residuals vs Predicted')
plt.xlabel('Predicted Eastward Wind')
plt.ylabel('Residuals')
plt.axhline(0, color='red', linewidth=2)
plt.show()


### Reference  
* ##### OpenAI. ChatGPT. https://openai.com/chatgpt  
* ##### Lecture Notes on Module 7-2-2 Artificial neural network II
* ##### Lecture Notes on Module 7-4-1 Regression model evaluation 
* #####  Yeturu, Jahnavi. (2019). Analysis of weather data using various regression algorithms. 117-141. 